In [1]:
#Import all the necessary libraries
import pymzml
import csv
import os
import pandas as pd
import numpy as np
import math
from matplotlib import pyplot as plt
import re
import plotly.express as px
from collections import defaultdict

import plotly.io as pio
import json
import plotly.graph_objs as go
import matplotlib.colors as mcolors

import json
import ipywidgets as widgets
from IPython.display import display

import warnings
from IPython.display import display, Image, clear_output
import time
##Custom scripts
from parsing_function import full_parse
from parsing_function import filter_dataframe
from parsing_function import hex_to_rgba_hex
from parsing_function import json_to_string
from parsing_function import prep_edge_R

from plotting_functions import make_pie_chart_no_replicates

from plotting_functions import average_pie_chart_no_repeats

from plotting_functions import make_bar_plot_comparisons

# dfdfplotting_functions

No module named 'ms_deisotope._c.averagine' averagine
No module named 'ms_deisotope._c.scoring'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'
No module named 'ms_deisotope._c.deconvoluter_base'


In [2]:


Pre_folder = './Projects/'

Project_Folder =Pre_folder + 'Burda_Lab/'
file_name_to_save = 'Burda_1_tolerance'

custom_data=True

tolerance = 0.1
remove_std = True
save_data= True


In [3]:

folder_name_to_save = Project_Folder+ 'Processed Results/'
data_base_name_location = 'lipid_database/Lipid_Database.xlsx'
mzml_folder = Project_Folder +"mzml/"
Pre_edge_r_path = Project_Folder+"Pre_EdgeR/"
plots_2_save_path = Project_Folder+"Plots/"
label_file = Project_Folder+"Labels/labels.csv"


In [4]:
##Labels DF and Labels List
labels_df = pd.read_csv(label_file)
labels_list = list(labels_df)
labels_list = labels_list +["Class","Lipid"]

In [7]:
# Assuming labels_df is your DataFrame and 'Sample Name' is the column with instances you want to select from
unique_samples = labels_df['Sample Name'].unique()

# Create a dropdown selection widget with instances as options
dropdown = widgets.Dropdown(
    options=unique_samples,
    value=unique_samples[0],  # default value
    description='Samples',
    disabled=False,
)
display(dropdown)

# Create a button to perform the assignment
button = widgets.Button(description="Assign Blank")
display(button)

# Output widget to display the selected sample name
output = widgets.Output()
display(output)

# Global variable to store the selected sample name
global blank_name

def on_button_clicked(b):
    # Declare blank_name as global variable
    global blank_name

    # Assign the selected sample name to blank_name
    blank_name = dropdown.value

    # Display the selected sample name
    with output:
        output.clear_output()
        print(f"Blank is: {blank_name}")

button.on_click(on_button_clicked)

Dropdown(description='Samples', options=('AK0161', 'AK0170', 'AK0139', 'AK0200', 'AK0202', 'AK0204', 'AK0174',…

Button(description='Assign Blank', style=ButtonStyle())

Output()

In [8]:

# Assuming df is your DataFrame and 'Sample name' is the column with instances you want to filter
unique_samples = labels_df['Sample Name'].unique()

# Create a multiple selection widget with instances as options
multi_select = widgets.SelectMultiple(
    options=unique_samples,
    value=[unique_samples[0]],  # default value
    rows=len(unique_samples),
    description='Samples',
    disabled=False
)
display(multi_select)

# Create a button to perform the filtering
button = widgets.Button(description="Filter Samples")
display(button)

# Output widget to display the resulting dataframe
output = widgets.Output()
display(output)

# Global variable to store the new DataFrame
global labels_df2

def on_button_clicked(b):
    # Declare labels_df2 as global variable
    global labels_df2
    
    # Clear the current output
    with output:
        output.clear_output()
        
    # Filter dataframe to exclude the selection
    labels_df2 = labels_df[~labels_df['Sample Name'].isin(multi_select.value)]
    
    # Display the new dataframe
    with output:
        display(labels_df2)

button.on_click(on_button_clicked)

SelectMultiple(description='Samples', index=(0,), options=('AK0161', 'AK0170', 'AK0139', 'AK0200', 'AK0202', '…

Button(description='Filter Samples', style=ButtonStyle())

Output()

In [9]:
labels_df2 = labels_df2.drop(["Sample Name","Position"],axis=1)

In [10]:
main_json = {col: labels_df2[col].unique().tolist() for col in labels_df2}

In [11]:
main_json

{'nickname': ['H_KO_1',
  'H_KO_2',
  'H_KO_3',
  'H_KO_5',
  'H_KO_6',
  'H_KO_7',
  'H_WT_1',
  'H_WT_2',
  'H_WT_3',
  'H_WT_4',
  'H_WT_6',
  'I_KO_1',
  'I_KO_2',
  'I_KO_3',
  'I_KO_4',
  'I_KO_5',
  'I_WT_1',
  'I_WT_2',
  'I_WT_3',
  'I_WT_4',
  'I_WT_5',
  'I_WT_6'],
 'Sex': ['M', 'F'],
 'Genotype': ['cKO', 'WT'],
 'time point': ['healthy', '28d ISCI']}

In [17]:
# Agent imports
from langchain.agents import create_json_agent, AgentExecutor, Tool
from langchain.agents.agent_toolkits import JsonToolkit
from langchain.agents.agent_toolkits.base import BaseToolkit

# Tool imports
from langchain.tools import BaseTool
from langchain.tools.human.tool import HumanInputRun
from langchain.tools.json.tool import JsonGetValueTool, JsonListKeysTool, JsonSpec
from langchain.tools.json.tool import JsonSpec

# LLM imports
from langchain.chains import LLMChain
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper

# Managers
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

# Typing + other imports
from typing import List, Optional, Callable
import json
import ast
from pathlib import Path

In [31]:
# Define custom tools
class JsonFilterFromKeysTool(BaseTool):
    

    def filter_json_nested(self,json_dict, keys_string, values_string):
        # Parse keys_string and values_string into lists
        keys = ast.literal_eval(keys_string)
        values = ast.literal_eval(values_string)

        filtered_dict = {}
        for k, v in json_dict.items():
            # If key and value are in the respective lists, add to filtered_dict
            if k in keys and v in values:
                filtered_dict[k] = v
            # If the value is a nested dictionary, filter the nested dictionary
            elif isinstance(v, dict):
                filtered_dict[k] = filter_json_nested(v, keys_string, values_string)
        return filtered_dict

    name="json_filter_json_from_lists"
    description = """Can be used to filter a JSON at a provided path based on provided keys and values.
        Before calling this you should be SURE that the path to this exists.
        The input is an exact text representation of two Python lists spearated by a semi colon where first list is the keys  to filter on, and the second list is the values to filter on.
    """
    spec: JsonSpec
    
    def _run(
        self,
        tool_input: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        return self.filter_json_nested(
            self.spec.dict_,
            tool_input.split(";")[0],
            tool_input.split(";")[1]
        )

    async def _arun(
        self,
        tool_input: str,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> str:
        return self._run(tool_input)

In [32]:

os.environ["OPENAI_API_KEY"] = ""


In [33]:
# Define our custom tool kit
json_spec = JsonSpec(dict_=main_json, max_value_length=4000)
#json_toolkit = JsonToolkit(spec=json_spec)

class CustomToolkit(BaseToolkit):
      
      spec: JsonSpec
      
      def get_tools(self) -> List[BaseTool]:
          return [
            JsonListKeysTool(spec=self.spec),
            JsonGetValueTool(spec=self.spec),
            JsonFilterFromKeysTool(spec=self.spec),
            HumanInputRun()
          ]

custom_toolkit = CustomToolkit(spec=json_spec)

In [ ]:
json_spec_list_keys  json_spec_get_value



In [51]:
# Define our prompt prefix and suffixes

prefix = """You are an agent designed to interact with a JSON and the user and have several potential objectives.
1. Provide the user with requested information on the JSON
2. Create a new JSON based off of user input and a filtered version of the original JSON

Your overall goal is to return a final answer by interacting with the user to create a new JSON object based on the original JSON.
You should talk like a cat when responding, make sure to use lots of meows and replace words that sounds like meow with meow.

You have access to the following tools which help you learn more about the JSON you are interacting with.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
Do not make up any information that is not contained in the JSON.

Your input to JSON tool filter_json_nested will be two python lists A list of dictionary keys and a list of dictionary 
values. The user should be asked for these lists with repeated questions for clarification.
Your input to JSON tools json_spec_list_keys and json_spec_get_value should be in the form of `data["key"][0]` where `data` is the JSON blob you are interacting with, and the syntax used is Python. 
You should only use keys that you know for a fact exist. You must validate that a key exists by seeing it previously when calling `json_spec_list_keys`. 
If you have not seen a key in one of those responses, you cannot use it.
You should only add one key at a time to the path. You cannot add multiple keys at once.
If you encounter a "KeyError", go back to the previous key, look at the available keys, and try again.



If the question does not seem to be related to the JSON, ask the user for guidance.
Always begin your interaction with the `json_spec_list_keys` tool with input "data" to see what keys exist in the JSON.
Always ask the user what they would like to do based on your objectives.
If you do not know which key to use or need any clarification ask the user for guidance.

Sometimes the keys the user provides will not be entirely correct.
You may need to infer what keys they mean.

Do not return your final answer until the user asks for it.

Note that sometimes the value at a given path is large. In this case, you will get an error "Value is a large dictionary, should explore its keys directly".
In this case, you should ALWAYS follow up by using the `json_spec_list_keys` tool to see what keys exist at that path.
Do not simply refer the user to the JSON or a section of the JSON, as this is not a valid answer. Keep digging until you find the answer and explicitly return it.
"""

suffix = """Begin!"

Remember to format your answers talking as a cat would.

Question: {input}
{agent_scratchpad}"""

In [52]:
# Build the agent executor
json_agent_executor = create_json_agent(
    llm=OpenAI(temperature=0),
    prefix=prefix,
    suffix=suffix,
    toolkit=custom_toolkit,
    verbose=True
)

In [53]:
# Initiate the agent executor run with an initial question
json_agent_executor.run("Hi, my name is Connor. Can you help me filter my JSON")



> Entering new AgentExecutor chain...
Thought: I need to figure out what keys and values to filter on
Action: json_spec_list_keys
Action Input: data
Observation: ['nickname', 'Sex', 'Genotype', 'time point']
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 6a01b211520242a2c4dac18de6af4c3f in your message.).


 I need to ask the user what keys and values they want to filter on
Action: Human
Action Input: What keys and values would you like to filter on?

What keys and values would you like to filter on?
Sex

Observation: Sex
Thought: I need to ask the user what values they want to filter on
Action: Human
Action Input: What values would you like to filter on?

What values would you like to filter on?
K

Observation: K
Thought: I now know the keys and values to filter on
Action: json_filter_json_from_keys
Action Input: ['Sex'], ['K']

IndexError: list index out of range

In [22]:
# List to hold pairs of JSON objects
json_list_pairs = []

def remove_empty_entries(json_list_pairs):
    cleaned_list_pairs = [
        [
            {key: value for key, value in pair_dict.items() if value} for pair_dict in pair
        ] for pair in json_list_pairs
    ]
    return cleaned_list_pairs

# Initialize widgets_dict1 and widgets_dict2 to be filled later
widgets_dict1 = {}
widgets_dict2 = {}

# Create a function that displays the pair widgets
def display_pair_widgets():
    global widgets_dict1, widgets_dict2
    widgets_dict1 = {key: widgets.SelectMultiple(options=value, description=key) for key, value in main_json.items()}
    widgets_dict2 = {key: widgets.SelectMultiple(options=value, description=key) for key, value in main_json.items()}
    
    for key in main_json.keys():
        display(widgets.HBox([widgets_dict1[key], widgets_dict2[key]]))

display_pair_widgets()

# Define what to do on 'Generate JSON files' button click
def on_generate_clicked(b):
    # Build new_json based on the values selected in the widgets
    new_json1 = {key: list(widget.value) for key, widget in widgets_dict1.items()}
    new_json2 = {key: list(widget.value) for key, widget in widgets_dict2.items()}

    # Add new JSON objects to the list
    pair = [new_json1, new_json2]
    json_list_pairs.append(pair)
    print("Complete")
    # Print the new JSON objects
#     print(json.dumps(new_json1, indent=2))
#     print(json.dumps(new_json2, indent=2))

def on_add_more_clicked(b):
    # Build new_json based on the values selected in the widgets
    new_json1 = {key: list(widget.value) for key, widget in widgets_dict1.items()}
    new_json2 = {key: list(widget.value) for key, widget in widgets_dict2.items()}

    # Add new JSON objects to the list
    pair = [new_json1, new_json2]
    json_list_pairs.append(pair)
    
    # Clear current selection
    for widget in widgets_dict1.values():
        widget.value = []
    for widget in widgets_dict2.values():
        widget.value = []

# Create the buttons
generate_button = widgets.Button(description='Finish')
generate_button.on_click(on_generate_clicked)

add_more_button = widgets.Button(description='Add more JSON pairs')
add_more_button.on_click(on_add_more_clicked)

# Display the buttons
display(widgets.HBox([generate_button, add_more_button]))

Complete


In [23]:
json_list_pairs = remove_empty_entries(json_list_pairs)


In [24]:
json_list_pairs

[[{'Sex': ['F'], 'Genotype': ['cKO']}, {'Sex': ['F'], 'Genotype': ['WT']}]]

In [ ]:
def get_unique_json_objects(json_list_pairs):
    json_set = set()
    for pair in json_list_pairs:
        for json_obj in pair:
            json_set.add(json.dumps(json_obj))
    
    json_list_singles = [json.loads(json_str) for json_str in json_set]
    return json_list_singles

In [ ]:
json_list_singles = get_unique_json_objects(json_list_pairs)